# Install Packgages

In [ ]:
!pip install fairseq
!pip install pythainlp
!pip install argostranslate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.6/9.6 MB 18.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 123.8/123.8 kB 14.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.7/106.7 kB 13.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 288.3/288.3 kB 29.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.4/112.4 kB 14.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia

# Import Library


In [ ]:
import pandas as pd
import torch
from PIL import Image
from transformers import AutoProcessor, AutoModelForCausalLM,  AutoModel, CLIPProcessor
from pythainlp.translate import Translate
import argostranslate.package
import argostranslate.translate

# Data

## Get Data from Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Image File

In [ ]:
# !unzip "/content/5_test_img.zip" -d "/content/testset/test2017"

In [ ]:
# image_root_path = "/content/testset/"
image_root_path = '/content/drive/MyDrive/Super_AI_Engineer_SS4/Level_2/Hackathon_1/6_test_img/'

## Submission File

In [ ]:
submission = pd.read_csv('/content/drive/MyDrive/Super_AI_Engineer_SS4/Level_2/Hackathon_1/5_sample_submission.csv')
submission.head()

,image_id,caption
0,test2017/000000000009,NaN
1,test2017/000000000025,NaN
2,test2017/000000000030,NaN
3,test2017/000000000034,NaN
4,test2017/000000000036,NaN


# Model

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cpu')

In [ ]:
processor = AutoProcessor.from_pretrained("microsoft/git-base-coco")
model = AutoModelForCausalLM.from_pretrained("microsoft/git-base-coco").to(device)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/503 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/453 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/2.82k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/707M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/141 [00:00<?, ?B/s]

## Generate Text

In [ ]:
def Image_to_Text(url):
    image = Image.open(url)
    pixel_values = processor(images = image, return_tensors = "pt").pixel_values.to(device) # Model Pre-Process data
    generated_ids = model.generate(pixel_values=pixel_values, max_length = 50)
    generated_caption = processor.batch_decode(generated_ids, skip_special_tokens = True)[0]
    return generated_caption

# Clean Text (Post-Process)

In [ ]:
def Clean_Text(text):
    text = text.replace("ค่ะ", "").strip()
    text = text.replace("(", "").strip()
    text = text.replace(")", "").strip()
    text = text.replace(".", "").strip()
    text = text.replace(" ", "").strip()
    text = text.replace('"', "").strip()
    text = text.replace("'", "").strip()
    return text

# Translate with PyThaiNLP

## Model

In [ ]:
translator = Translate('en', 'th')

def PyThai_Translate(text):
    text = translator.translate(text)
    return text

Corpus: scb_1m_en-th_moses
- Downloading: scb_1m_en-th_moses 1.0


  0%|          | 0/1174648148 [00:00<?, ?it/s]

## Result

In [ ]:
pythai_submission = submission

In [ ]:
for i in range(len(submission)): # set start == 3 if use sample_submission.csv file
    url = image_root_path + submission["image_id"][i] + ".jpg"
    print(submission["image_id"][i])
    caption = Image_to_Text(url)
    tran_cap = PyThai_Translate(caption)
    clean_cap = Clean_Text(tran_cap)
    print("{} -> {}".format(caption, clean_cap))
    submission["caption"][i] = clean_cap

test2017/000000000009
a tray of food with broccoli and fruit. -> ถาดอาหารที่มีบร็อคโคลี่กับผลไม้
test2017/000000000025
a giraffe standing next to a tree in a forest. -> ยีราฟยืนอยู่ข้างต้นไม้ในป่า
test2017/000000000030
a vase with flowers on a white table. -> แจกันที่มีดอกไม้บนโต๊ะสีขาว
test2017/000000000034
a zebra is grazing on the grass in the sun. -> ม้าลายเป็นพื้นหญ้าในดวงอาทิตย์
test2017/000000000036
a woman in a swimsuit holding a pink umbrella. -> ผู้หญิงในชุดว่ายน้ําถือร่มสีชมพู
test2017/000000000042
a dog laying in a basket with a pair of shoes. -> สุนัขนอนอยู่ในตะกร้ากับรองเท้าคู่หนึ่ง


In [ ]:
submission.head()

,image_id,caption
0,test2017/000000000009,ถาดอาหารที่มีบร็อคโคลี่กับผลไม้
1,test2017/000000000025,ยีราฟยืนอยู่ข้างต้นไม้ในป่า
2,test2017/000000000030,แจกันที่มีดอกไม้บนโต๊ะสีขาว
3,test2017/000000000034,ม้าลายเป็นพื้นหญ้าในดวงอาทิตย์
4,test2017/000000000036,ผู้หญิงในชุดว่ายน้ําถือร่มสีชมพู


## Save Result

In [ ]:
submission.to_csv('PyThai_submission.csv', index = False)

# Translate with ArgosTranslate

## Model

In [ ]:
from_code = "en"
to_code = "th"

# Download and install Argos Translate package
argostranslate.package.update_package_index()
available_packages = argostranslate.package.get_available_packages()
package_to_install = next(
    filter(
        lambda x: x.from_code == from_code and x.to_code == to_code, available_packages
    )
)
argostranslate.package.install_from_path(package_to_install.download())

## Result

In [ ]:
argos_submission = submission

In [ ]:
for i in range(len(submission)): # set start == 3 if use sample_submission.csv file
    url = image_root_path + submission["image_id"][i] + ".jpg"
    print(submission["image_id"][i])
    caption = Image_to_Text(url)
    tran_cap = argostranslate.translate.translate(caption, from_code, to_code)
    clean_cap = Clean_Text(tran_cap)
    print("{} -> {}".format(caption, clean_cap))
    submission["caption"][i] = clean_cap

test2017/000000000009
a tray of food with broccoli and fruit. -> เป็นถาดอาหารของบร็อคโคลี่และผลไม้
test2017/000000000025
a giraffe standing next to a tree in a forest. -> ยีราฟยืนข้างต้นไม้ในป่า
test2017/000000000030
a vase with flowers on a white table. -> แจกันที่มีดอกไม้บนโต๊ะสีขาว
test2017/000000000034
a zebra is grazing on the grass in the sun. -> ม้าลายกําลังกินหญ้ากลางแดด
test2017/000000000036
a woman in a swimsuit holding a pink umbrella. -> ผู้หญิงในชุดว่ายน้ําถือร่มสีชมพู
test2017/000000000042
a dog laying in a basket with a pair of shoes. -> สุนัขที่นอนในตะกร้ากับรองเท้าคู่หนึ่ง


In [ ]:
submission

,image_id,caption
0,test2017/000000000009,เป็นถาดอาหารของบร็อคโคลี่และผลไม้
1,test2017/000000000025,ยีราฟยืนข้างต้นไม้ในป่า
2,test2017/000000000030,แจกันที่มีดอกไม้บนโต๊ะสีขาว
3,test2017/000000000034,ม้าลายกําลังกินหญ้ากลางแดด
4,test2017/000000000036,ผู้หญิงในชุดว่ายน้ําถือร่มสีชมพู
5,test2017/000000000042,สุนัขที่นอนในตะกร้ากับรองเท้าคู่หนึ่ง


## Save Result

In [ ]:
submission.to_csv('Argos_submission.csv', index = False)